<a href="https://colab.research.google.com/github/chuanxiangpei/MyProject/blob/master/2022_CCF_BDCI_%E8%BF%94%E4%B9%A1%E5%8F%91%E5%B1%95%E4%BA%BA%E7%BE%A4%E9%A2%84%E6%B5%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/data"

os.chdir(path)
os.listdir(path)

print()
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

dataA.csv  dataTrain.csv  models  pred.csv


1. 导入包

In [ ]:
# -*- coding: utf-8 -*-
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

import torch.nn as nn
import torchvision.transforms as transforms

# sklearn package
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import  SelectFromModel

2. 超参数

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. 
    # 'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # valid_size = train_size * valid_ratio
    'test_ratio': 0.2,
    'n_epochs': 100,     # Number of epochs.            
    'batch_size': 128, 
    'learning_rate': 1e-3,              
    'hidden_layers': 10, 
    'hidden_dim': 512,
    'early_stop': 50,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

3. 随机性

In [ ]:
def same_seed(seed):
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

4. 数据封装

In [ ]:
class MyDataset(Dataset):
  def __init__(self, x, y=None):
    self.x = torch.FloatTensor(x)
    if y is None:
      self.y = y
    else:
      self.y = torch.FloatTensor(y)

  def __getitem__(self, index):
    if self.y is None:
      return self.x[index]
    else:
      return self.x[index], self.y[index]

  def __len__(self):
    return len(self.x)

5. 模型设计

In [ ]:
class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
            nn.BatchNorm1d(output_dim),
            nn.Dropout(0.2)   # 正则
        )

    def forward(self, x):
        x = self.block(x)
        return x


class MyModel(nn.Module):
    def __init__(self, input_dim, output_dim=2, hidden_layers=3, hidden_dim=200):
        super(MyModel, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

In [ ]:
np.set_printoptions(threshold=np.inf) 
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

6. 数据导入与数据预处理

In [ ]:
# type: DataFrame
# test_data_df = pd.read_csv('dataA.csv')
train_data_df = pd.read_csv('dataTrain.csv')

# print(f"""train_data df:{train_data_df.shape}
#       test_data df:{test_data_df.shape}""")

# transform column(f3) to digits
f3_mapping = {'low':0, 'mid':1, 'high':2}
train_data_df['f3'] = train_data_df['f3'].map(f3_mapping)
# test_data_df['f3'] = test_data_df['f3'].map(f3_mapping)


# Return a Numpy representation of the DataFrame.
train_data = train_data_df.values
# test_data = test_data_df.values

7. 特征提取及数据封装

In [ ]:
same_seed(config['seed'])


# Data cleansing
# Removing interfering data
train_data = train_data[:50000]


# spilit train_data to train_data and valid_data
# train_data, valid_data = train_valid_split(train_data, config['train_ratio'], config['seed'])

X_train, X_test, y_train, y_test = train_test_split(train_data[:, 1:-1], train_data[:, -1], test_size=config['test_ratio'], random_state=config['seed'], stratify=train_data[:, -1])
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=config['valid_ratio'], random_state=config['seed'], stratify=y_train)

print(f"""X_train size:{X_train.shape}
      X_valid size:{X_valid.shape}
      X_test size:{X_test.shape}""")

# 选取某些特征
# 基于哪个模型
etc = ExtraTreesClassifier()

# 用这个模型训练数据
etc_ = etc.fit(X_train, y_train)

# 得到各个特征的重要性参数
feature_importances = etc_.feature_importances_

# 列表形式得到n个相对重要性参数的序号
idx = np.argsort(feature_importances)

# 根据模型选择特征
select_from_model = SelectFromModel(etc_, prefit=True)

# 将x_train进行transform
X_SFM_ETC = select_from_model.transform(X_train)

# 得到feature_idx
feature_idx = idx[-X_SFM_ETC.shape[1]:]
print(f'feature_idx:{feature_idx}')


X_train, X_valid, X_test = X_train[:,feature_idx], X_valid[:,feature_idx], X_test[:,feature_idx]

# x_train, x_valid, x_test, y_train, y_valid = select_feature(train_data, valid_data, test_data, config['select_all'])
print(f'number of features:{X_train.shape[1]}')
print(f"""X_train size:{X_train.shape}
      X_valid size:{X_valid.shape}
      X_test size:{X_test.shape}
      y_train size:{y_train.shape}
      y_valid size:{y_valid.shape}""")


train_dataset = MyDataset(X_train, y_train)

valid_dataset = MyDataset(X_valid, y_valid)

test_dataset = MyDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)


X_train size:(32000, 46)
      X_valid size:(8000, 46)
      X_test size:(10000, 46)
feature_idx:[42 23 43 22 36 18 24 25 44 45  0  1  4  5  3]
number of features:15
X_train size:(32000, 15)
      X_valid size:(8000, 15)
      X_test size:(10000, 15)
      y_train size:(32000,)
      y_valid size:(8000,)


8. 模型训练

In [ ]:
model = MyModel(input_dim=X_train.shape[1], hidden_layers=config['hidden_layers'], hidden_dim=config['hidden_dim']).to(device)
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.9, weight_decay=1e-5)
optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate'])
scheduler = CosineAnnealingWarmRestarts(optimizer,T_0=5,T_mult=1)  # T_mult=2时效果不好，准确率不高

if not os.path.isdir('./models'):
    os.mkdir('./models')

n_epochs = config['n_epochs']
best_acc = 0.0
early_stop_count = 0

for epoch in range(n_epochs):
    # train
    model.train()
    loss_record = []
    acc_record = []
    train_pbar = tqdm(train_loader)
    for X, y in train_pbar:
      optimizer.zero_grad()
      X, y = X.to(device), y.to(device)
      pred = model(X)
      loss = criterion(pred, y.long())  # y.long() is equivalent to self.to(torch.int64)
      loss.backward()
      
      # Clip the gradient norms for stable training.
      grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)
      
      optimizer.step()
      scheduler.step()   
      
      loss_record.append(loss.detach().item())
      train_pbar.set_description(f'Epoch  [{epoch+1}/{n_epochs}]')
      train_pbar.set_postfix({'loss': loss.detach().item()})
   
      _, pred_max_index = torch.max(pred.data, dim=1) 
      acc = (pred_max_index.detach() == y.detach()).float().mean().item() # before add .item():<class 'torch.Tensor'>    after add .item():<class 'float'>
      acc_record.append(acc)

    mean_train_loss = sum(loss_record) / len(loss_record)
    # torch.argmax(dim=0):column, torch.argmax(dim=1):row
    mean_train_acc = sum(acc_record) / len(acc_record)
    
    print(f'Epoch  [{epoch+1} / {n_epochs}]:  Train Loss:{mean_train_loss:.4f}, Train Acc:{mean_train_acc}')

    
    # valid
    model.eval()
    valid_loss_record = []
    valid_acc_record = []
    for X, y in valid_loader:
      X, y = X.to(device), y.to(device)
      with torch.no_grad():
        pred = model(X)
      loss = criterion(pred, y.long())
      valid_loss_record.append(loss.item())
    
      _, pred_max_index = torch.max(pred, dim=1) 
      acc = (pred_max_index.detach() == y.detach()).float().mean().item()
      valid_acc_record.append(acc)

    mean_valid_loss = sum(valid_loss_record) / len(valid_loss_record)
    mean_valid_acc = sum(valid_acc_record) / len(valid_acc_record)
    
    
    print(f'Epoch  [{epoch+1} / {n_epochs}]:  Valid Loss:{mean_valid_loss:.4f}, Valid Acc:{mean_valid_acc}')
    
    if mean_valid_acc > best_acc:
      best_acc = mean_valid_acc
      torch.save(model.state_dict(), config['save_path'])
      print(f'Saving model with acc {best_acc:.3f}, Best model found at epoch {epoch}')
      early_stop_count = 0
    else:
      early_stop_count += 1
      if early_stop_count >= config['early_stop']:
          print('Model is not improving, so we halt the traing session')
          break



Epoch  [1/100]: 100%|██████████| 250/250 [00:21<00:00, 11.69it/s, loss=0.546]


Epoch  [1 / 100]:  Train Loss:0.5237, Train Acc:0.76128125
Epoch  [1 / 100]:  Valid Loss:0.4088, Valid Acc:0.8328373015873016
Saving model with acc 0.833, Best model found at epoch 0


Epoch  [2/100]: 100%|██████████| 250/250 [00:20<00:00, 12.27it/s, loss=0.352]


Epoch  [2 / 100]:  Train Loss:0.4118, Train Acc:0.82853125
Epoch  [2 / 100]:  Valid Loss:0.3706, Valid Acc:0.8509424603174603
Saving model with acc 0.851, Best model found at epoch 1


Epoch  [3/100]: 100%|██████████| 250/250 [00:20<00:00, 12.24it/s, loss=0.374]


...

Epoch  [94 / 100]:  Train Loss:0.2836, Train Acc:0.877625
Epoch  [94 / 100]:  Valid Loss:0.3476, Valid Acc:0.8624751984126984


Epoch  [95/100]: 100%|██████████| 250/250 [00:20<00:00, 12.03it/s, loss=0.247]


Epoch  [95 / 100]:  Train Loss:0.2824, Train Acc:0.88015625
Epoch  [95 / 100]:  Valid Loss:0.3556, Valid Acc:0.8628472222222222


Epoch  [96/100]: 100%|██████████| 250/250 [00:20<00:00, 12.09it/s, loss=0.291]


Epoch  [96 / 100]:  Train Loss:0.2793, Train Acc:0.87965625
Epoch  [96 / 100]:  Valid Loss:0.3547, Valid Acc:0.8612351190476191
Model is not improving, so we halt the traing session


9. 对测试数据进行预测

In [ ]:
def save_pred(preds, file):
  with open(file, 'w') as fp:
    writer = csv.writer(fp)
    writer.writerow(['id', 'label'])
    for i, p in enumerate(preds):
      writer.writerow([i, p])

In [ ]:
model = MyModel(input_dim=X_train.shape[1], hidden_layers=config['hidden_layers'], hidden_dim=config['hidden_dim']).to(device)
model.load_state_dict(torch.load(config['save_path']))
model.eval()
test_acc_record = []
prediction = []
for X, y in test_loader:
  X = X.to(device)
  y = y.to(device)
  with torch.no_grad():
    pred = model(X)
  # torch.max:依次最大值和最大值的索引，dim=1（按列返回）
  _, idx_max = torch.max(pred.cpu().data, dim=1)
  acc = (idx_max.detach() == y.detach()).float().mean().item()
  test_acc_record.append(acc)  
  prediction.extend(idx_max.tolist())
mean_test_acc = sum(test_acc_record) / len(test_acc_record)
print(f'Epoch  [{epoch+1} / {n_epochs}]:  test Acc:{mean_test_acc}')

preds = np.array(prediction)
save_pred(preds, 'pred.csv')

Epoch  [96 / 100]:  test Acc:0.8581882911392406
